---
title: Fractional gates
description: Use fractional gates to execute single- and two-qubit rotations
---


# Fractional gates

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>

This page introduces two newly supported gate types on the IBM Quantum&reg; fleet of QPUs. These *fractional* gates are supported on [Heron QPUs](/docs/guides/processor-types#heron) in the form of:
- $R_{ZZ}(\theta)$ for $0 \lt \theta \leq \pi/2$
- $R_X(\theta)$ for any $\theta$

This page discusses the use cases where implementing fractional gates can improve the efficiency of your workflows, as well as how to use these gates on IBM Quantum QPUs.

## How to use fractional gates

Internally, these fractional gates work by directly executing an $R_{ZZ}(\theta)$ and $R_X(\theta)$ rotation for an arbitrary angle. Use of the $R_X(\theta)$ gate can reduce the duration and error for single-qubit rotations of arbitrary angle by up to a factor of two. The direct execution of the $R_{ZZ}(\theta)$ gate rotation avoids decomposition into multiple `CZGate` objects, similarly reducing a circuit's duration and error. This is especially useful for circuits that contain many single- and two-qubit rotations, such as when simulating the dynamics of a quantum system or when using a variational ansatz with many parameters.


While these types of gates are in the [library of standard gates](./circuit-library) which a `QuantumCircuit` can possess, they can only be used on specific IBM Quantum QPUs, and which must be loaded with the flag `use_fractional_gates` set to `True` (shown below). This flag will ensure that fractional gates are included in the backend's `Target` for the transpiler.

```python
service = QiskitRuntimeService()
backend = service.backend('ibm_torino', use_fractional_gates=True)
```


This code example demonstrates how to use fractional gates in the context of a workflow that simulates the dynamics of an Ising chain using fractional gates. The circuit duration is then compared against a backend that does not use fractional gates.

<Admonition title="Note about reported error rates" type="note">
  The error value reported in the `Target` of a backend with fractional gates enabled is just a copy of the non-fractional gate's counterpart (which may not be the same). This is because the reporting of error rates on the fractional gates is not yet supported.

  However, since the gate time of fractional versus non-fractional gates are the same, it is a reasonable assumption that their error rates are comparable -- especially when the dominant source of error in a circuit is due to relaxation.
</Admonition>

In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization.timeline import draw as draw_timeline, IQXSimple

from qiskit_ibm_runtime import QiskitRuntimeService


num_qubits = 5
num_time_steps = 3
rx_angle = 0.1
rzz_angle = 0.1

ising_circuit = QuantumCircuit(num_qubits)
for i in range(num_time_steps):
    # rx layer
    for q in range(num_qubits):
        ising_circuit.rx(rx_angle, q)
    for q in range(1, num_qubits - 1, 2):
        ising_circuit.rzz(rzz_angle, q, q + 1)
    # 2nd rzz layer
    for q in range(0, num_qubits - 1, 2):
        ising_circuit.rzz(rzz_angle, q, q + 1)
    ising_circuit.barrier()
ising_circuit.draw("mpl")

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/9cb7f696-dec0-4393-9320-fe945326c893-0.svg" alt="Output of the previous code cell" />

Specify two backend objects: one with fractional gates enabled, and the other with them disabled, then transpile them both.

In [2]:
service = QiskitRuntimeService()
backend_fractional = service.backend("ibm_torino", use_fractional_gates=True)
backend_conventional = service.backend(
    "ibm_torino", use_fractional_gates=False
)

pm_fractional = generate_preset_pass_manager(
    optimization_level=3, backend=backend_fractional, scheduling_method="alap"
)
pm_conventional = generate_preset_pass_manager(
    optimization_level=3,
    backend=backend_conventional,
    scheduling_method="alap",
)

ising_circuit_fractional = pm_fractional.run(ising_circuit)
ising_circuit_conventional = pm_conventional.run(ising_circuit)

Display the timeline of the circuit using the two types of gates.

In [3]:
# Draw timeline of circuit with conventional gates
draw_timeline(
    ising_circuit_conventional,
    idle_wires=False,
    target=backend_conventional.target,
    time_range=(0, 500),
    style=IQXSimple(),
)

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/0013f5fa-4072-4aa4-94fe-7e195435f828-0.svg" alt="Output of the previous code cell" />

In [4]:
# Draw timeline of circuit with fractional gates
draw_timeline(
    ising_circuit_fractional,
    idle_wires=False,
    target=backend_fractional.target,
    time_range=(0, 500),
    style=IQXSimple(),
)

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/08dd1cdf-8b34-47c2-8324-f3538c9d1ab6-0.svg" alt="Output of the previous code cell" />

### Angle constraints

For the two-qubit $R_{ZZ}(\theta)$ gate, only angles between $0$ and $\pi/2$ can be executed on IBM Quantum hardware. If a circuit contains any $R_{ZZ}(\theta)$ gates with an angle outside of this range, then the standard transpilation pipeline will generally correct this with an appropriate circuit transformation (through the [`FoldRzzAngle`](../api/qiskit-ibm-runtime/transpiler-passes-fold-rzz-angle) pass).  However, for any $R_{ZZ}(\theta)$ gate containing one or more [`Parameter`](../api/qiskit/qiskit.circuit.Parameter)s, the transpiler will assume these parameters will be assigned angles within this range at runtime. The job will fail if any of the parameter values specified in the PUB submitted to Qiskit Runtime are outside of this range.

## Where to use fractional gates

Historically, the basis gates available on IBM Quantum QPUs have been **`CZ`**, **`X`**, **`RZ`**, **`SX`**, and **`ID`**, which can not efficiently represent circuits with single- and two-qubit rotations that are not multiples of $\pi / 2$. For example, an $R_X(\theta)$ gate, when transpiled, must decompose into a series of $RZ$ and $\sqrt{X}$ gates, which creates a circuit with two gates of finite duration instead of one.

Similarly, when two-qubit rotations such as an $R_{ZZ}(\theta)$ gate are transpiled, the decomposition requires two **`CZ`** gates and several single-qubit gates, which increases the circuit depth. These decompositions are shown in the following code.

In [5]:
qc = QuantumCircuit(1)
param = Parameter("θ")
qc.rx(param, 0)
qc.draw("mpl")

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/11b003ee-aa8e-4794-a84a-b6870d15fa11-0.svg" alt="Output of the previous code cell" />

In [6]:
# Decomposition of an RX(θ) gate using the IBM Quantum QPU basis
service = QiskitRuntimeService()
backend = service.backend("ibm_torino")
optimization_level = 3
pm = generate_preset_pass_manager(optimization_level, backend=backend)
transpiled_circuit = pm.run(qc)
transpiled_circuit.draw("mpl", idle_wires=False)

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/5da9bba4-9a3b-4569-9997-c5b9ccf87b6a-0.svg" alt="Output of the previous code cell" />

In [7]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.transpiler import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService

qc = QuantumCircuit(2)
param = Parameter("θ")
qc.rzz(param, 0, 1)
qc.draw("mpl")

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/6b256201-0237-4f63-91ff-fde1bb884804-0.svg" alt="Output of the previous code cell" />

In [8]:
# Decomposition of an RZZ(θ) gate using the IBM Quantum QPU basis
service = QiskitRuntimeService()
backend = service.backend("ibm_torino")
optimization_level = 3
pm = generate_preset_pass_manager(optimization_level, backend=backend)
transpiled_circuit = pm.run(qc)
transpiled_circuit.draw("mpl", idle_wires=False)

<Image src="/docs/images/guides/fractional-gates/extracted-outputs/f07217b9-a6f0-4adf-b341-6da447535c33-0.svg" alt="Output of the previous code cell" />

For workflows that require many single-qubit $R_X(\theta)$ or two-qubit rotations (such as in a variational ansatz or when simulating the time evolution of quantum systems), this constraint causes the circuit depth to grow rapidly. However, fractional gates remove this requirement, because the single- and two-qubit rotations are executed directly, and create a more efficient (and thus error-suppressed) quantum circuit.

<span id="when-not-to-use"></span>
## When *not* to use fractional gates

It is important to note that fractional gates are an *experimental* feature and the behavior of the `use_fractional_gates` flag may change in the future. Look to the [release notes](/docs/api/qiskit-ibm-runtime/release-notes) for new versions of Qiskit Runtime for more information. See also the API reference documentation for [QiskitRuntimeService.backend](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#backend), which describes `use_fractional_gates`.

Additionally, the Qiskit transpiler has limited capability to use $R_{ZZ}(\theta)$ in its optimization passes. This requires you to take more care in crafting and optimizing circuits that contain these instructions.

Lastly, using fractional gates is not supported for:
- [Dynamic circuits](/docs/guides/classical-feedforward-and-control-flow)
- [Pauli twirling](/docs/guides/error-mitigation-and-suppression-techniques#pauli-twirling) - however, [measurement twirling with TREX](/docs/guides/error-mitigation-and-suppression-techniques#twirled-readout-error-extinction-trex) *is* supported.
- [Probabilistic error cancellation](/docs/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec)
- [Zero-noise extrapolation (using probabilistic error amplification)](/docs/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea)

Read the guide on [primitive options](/docs/guides/runtime-options-overview) to learn more about customizing the error mitigation and suppression techniques for a given quantum workload.

## Next steps

<Admonition type="tip" title="Recommendations">
  -  To learn more about transpilation, see the [introduction to transpilation](/docs/guides/transpile) page.
  -  Read about [writing a custom transpiler pass](/docs/guides/custom-transpiler-pass).
  -  Understand how to [configure error mitigation](/docs/guides/configure-error-mitigation) for Qiskit Runtime.
</Admonition>